In [1]:
import os
'''
os.environ["JAX_PLATFORMS"] = "cpu"
os.environ["CUDA_VISIBLE_DEVICES"] = ""
os.environ["XLA_FLAGS"] = "--xla_cpu_multi_thread_eigen=true intra_op_parallelism_threads=128"
'''
import jax
print(jax.devices())

[CudaDevice(id=0), CudaDevice(id=1), CudaDevice(id=2), CudaDevice(id=3)]


In [2]:
import jax
import jax.numpy as jnp
import SSjax
import NSjax
import NNjax
import h5py
import json
import time
import numpy
import pyccl
import scipy
from itertools import product
from matplotlib import pyplot
from matplotlib.gridspec import GridSpec
from matplotlib.ticker import FixedLocator, FixedFormatter
import matplotlib.lines as mlines
import multiprocessing
import psutil
import threading

In [3]:
# Path
folder = '/global/cfs/cdirs/lsst/groups/MCP/CosmoCloud/SOMZCloud/'
cell_folder = os.path.join(folder, 'CELL/')
info_folder = os.path.join(folder, 'INFO/')
model_folder = os.path.join(folder, 'MODEL/')
tag='Y1'
typ='FIDUCIAL'
label='DOUBLE'
name='SS'
'''
synthesize_folder = os.path.join(folder, 'SYNTHESIZE/')

tag='Y1'
typ='FIDUCIAL'
label='DOUBLE'
name='SS'

os.makedirs(os.path.join(cell_folder, '{}/'.format(tag)), exist_ok = True)
os.makedirs(os.path.join(cell_folder, '{}/{}'.format(tag, name)), exist_ok = True)
'''

"\nsynthesize_folder = os.path.join(folder, 'SYNTHESIZE/')\n\ntag='Y1'\ntyp='FIDUCIAL'\nlabel='DOUBLE'\nname='SS'\n\nos.makedirs(os.path.join(cell_folder, '{}/'.format(tag)), exist_ok = True)\nos.makedirs(os.path.join(cell_folder, '{}/{}'.format(tag, name)), exist_ok = True)\n"

In [4]:
# Load data
with h5py.File('/global/cfs/cdirs/lsst/groups/MCP/CosmoCloud/SOMZCloud/CALIBRATE/{}/CORRECT/TITANIUM/HYBRID.hdf5'.format(tag), 'r') as file:
    data_source = file['source']['data'][...]
data_size, bin_source_size, z_size = data_source.shape

# Alignment
with open(os.path.join(info_folder, 'ALIGNMENT.json'), 'r') as file:
    alignment_info = json.load(file)

In [5]:
# Redshift
z1 = 0.0
z2 = 3.0
z_grid = numpy.linspace(z1, z2, z_size)

In [6]:
# Cosmology
with open(os.path.join(info_folder, 'COSMOLOGY.json'), 'r') as file:
    cosmology_info = json.load(file)

Omega_m = cosmology_info['OMEGA_M']

cosmology = pyccl.Cosmology(
    h=cosmology_info['H'],
    w0=cosmology_info['W0'],
    wa=cosmology_info['WA'],
    n_s=cosmology_info['NS'], 
    A_s=cosmology_info['AS'], 
    m_nu=cosmology_info['M_NU'],
    Neff=cosmology_info['N_EFF'],
    Omega_k=cosmology_info['OMEGA_K'], 
    Omega_b=cosmology_info['OMEGA_B'], 
    Omega_c=cosmology_info['OMEGA_CDM'],
    Omega_g=cosmology_info['OMEGA_GAMMA'], 
    mass_split = 'single', matter_power_spectrum = 'halofit', transfer_function = 'boltzmann_camb',
    extra_parameters = {'camb': {'kmax': 100, 'lmax': 5000, 'halofit_version': 'mead2020_feedback', 'HMCode_logT_AGN': 7.8}}
)

    
pyccl.spline_params.N_K = z_size
pyccl.gsl_params.NZ_NORM_SPLINE_INTEGRATION = True

In [7]:
# Multipole
ell1 = 20
ell2 = 2000
ell_size = 20
ell_grid = numpy.geomspace(ell1, ell2, ell_size + 1)
ell_data = numpy.sqrt(ell_grid[+1:] * ell_grid[:-1])

A_GRID = numpy.array(1 / (1 + z_grid), dtype = 'double')

CHI_GRID = cosmology.comoving_radial_distance(a = A_GRID)
SOURCE_PHI_GRID = data_source * cosmology.h_over_h0(a = 1 / (1 + z_grid)) * cosmology_info['H'] * 100000 / scipy.constants.c

CHI_MESH, ELL_MESH = numpy.meshgrid(CHI_GRID, ell_grid)
SCALE_GRID = numpy.nan_to_num(numpy.divide(ELL_MESH + 1/2, CHI_MESH, out = numpy.zeros((ell_size + 1, z_size)) + numpy.inf, where = CHI_MESH > 0))
POWER_GRID = numpy.zeros((ell_size + 1, z_size))
T0 = time.time()
for GRID_INDEX in range(z_size):

    POWER_GRID[:,GRID_INDEX] = cosmology.nonlin_matter_power(k = SCALE_GRID[:,GRID_INDEX], a = A_GRID[GRID_INDEX])
T1 = time.time()
T_POWER=T1-T0
AMPLITUDE = 9 / 4 * Omega_m**2 * (cosmology_info['H'] * 100000 / scipy.constants.c)**4 * (1 + 3 / (2 * ell_grid + 1)) * (1 + 1 / (2 * ell_grid + 1)) * (1 - 1 / (2 * ell_grid + 1)) * (1 - 3 / (2 * ell_grid + 1))
AMPLITUDE_IS = 3 / 2 * Omega_m * (cosmology_info['H'] * 100000 / scipy.constants.c)**2 * numpy.sqrt((1 + 3 / (2 * ell_grid + 1)) * (1 + 1 / (2 * ell_grid + 1)) * (1 - 1 / (2 * ell_grid + 1)) * (1 - 3 / (2 * ell_grid + 1)))
AMPLITUDE_II = 1
print(T_POWER)

1.286355972290039


In [8]:
#Angular power Spectra(CCL)
Number_of_round=500
C_CCL = numpy.zeros((Number_of_round, bin_source_size, bin_source_size, ell_size+1))
    
l_correct=numpy.sqrt((1 + 3 / (2 * ell_grid + 1)) * (1 + 1 / (2 * ell_grid + 1)) * (1 - 1 / (2 * ell_grid + 1)) * (1 - 3 / (2 * ell_grid + 1)))
A = numpy.asarray(alignment_info['A'])
POWER_GRID_IS = POWER_GRID * A
POWER_GRID_II = POWER_GRID * A * A
T_pyccl=numpy.zeros(Number_of_round)
for (r, i, j) in product(range(Number_of_round), range(bin_source_size), range(bin_source_size)):
    T0 = time.time()
    tracer1 = pyccl.tracers.WeakLensingTracer(cosmo=cosmology, dndz=[z_grid, data_source[r, i, :]], has_shear=True, ia_bias=[z_grid, alignment_info['A']], use_A_ia=False, n_samples=z_size)
    tracer2 = pyccl.tracers.WeakLensingTracer(cosmo=cosmology, dndz=[z_grid, data_source[r, j, :]], has_shear=True, ia_bias=[z_grid, alignment_info['A']], use_A_ia=False, n_samples=z_size)
    C_CCL[r, i, j,:] = pyccl.cells.angular_cl(cosmo=cosmology, tracer1=tracer1, tracer2=tracer2, ell=ell_grid, p_of_k_a='delta_matter:delta_matter', l_limber=-1, limber_max_error=0.01, limber_integration_method='qag_quad', non_limber_integration_method='FKEM', fkem_chi_min=0, fkem_Nchi=z_size, p_of_k_a_lin='delta_matter:delta_matter', return_meta=False)
    T1 = time.time()
    T_pyccl[r]=T_pyccl[r]+T1-T0
    
print(T_pyccl)
numpy.savetxt("T_pyccl.txt", T_pyccl, fmt="%.8e")

[0.08461642 0.12067652 0.07117939 0.0658927  0.32121277 0.10118532
 0.30378318 0.0677979  0.0685358  0.12300849 0.06618857 0.06802678
 0.06691861 0.16094017 0.06768584 0.06420732 0.0667603  0.06965232
 0.13888121 0.06951451 0.07034945 0.06846046 0.13432384 0.06616664
 0.10680842 0.06966972 0.32922864 0.22753072 0.06837416 0.13037181
 0.06632161 0.23213005 0.12069249 0.06857872 0.06796336 0.07131505
 0.19004202 0.06991124 0.06650257 0.06920075 0.16875839 0.06626225
 0.06794095 0.06781054 0.12644458 0.06730461 0.07732368 0.06707883
 0.06625533 0.12875366 0.06619024 0.066926   0.28680301 0.06929517
 0.12793255 0.12683368 0.06845069 0.06657648 0.06807017 0.1232264
 0.07226133 0.13419819 0.06759834 0.12352133 0.06889772 0.06814075
 0.0884347  0.11878204 0.07046223 0.06834555 0.07191324 0.06797361
 0.18261671 0.0708735  0.06769013 0.06843495 0.18644762 0.07521343
 0.06575394 0.12513614 0.06921029 0.0691328  0.0712986  0.1033957
 0.19127774 0.06718779 0.07348204 0.1619451  0.06735086 0.067671

In [ ]:
#Angular power Spectra(TensorCloud)
POWER_A   = POWER_GRID * A
POWER_A2  = POWER_GRID * (A * A)

def _one_round(phi1,phi2):
    
    SS = SSjax.function(AMPLITUDE,  phi1, phi2, CHI_GRID, POWER_GRID,  z_grid)              
    II = (l_correct * l_correct) * NNjax.function(AMPLITUDE_II, phi1, phi2, CHI_GRID, POWER_A2 , z_grid)
    IS =  l_correct * NSjax.function(AMPLITUDE_IS, phi1, phi2, CHI_GRID, POWER_A,  z_grid)
    SI = jnp.swapaxes(IS, 0, 1)  

    return SS + IS + SI + II  

batched_fn = jax.jit(jax.vmap(_one_round, in_axes=(0,0)))    

#Testing different batch sizes
batch_sizes = [int(Number_of_round/5), int(Number_of_round/5*2), int(Number_of_round/5*3), int(Number_of_round/5*4), Number_of_round]
timing_results = []  

for batch_size in batch_sizes:
    # Different batch sizes

    SOURCE_PHI_GRID_batch = SOURCE_PHI_GRID[:batch_size]
    SOURCE_PHI_GRID_j = jnp.asarray(SOURCE_PHI_GRID_batch)
    
    # Pre-heat
    _ = batched_fn(SOURCE_PHI_GRID_j, SOURCE_PHI_GRID_j)
    _.block_until_ready()
    
    #Timing
    t0 = time.time()
    C_DEV = batched_fn(SOURCE_PHI_GRID_j, SOURCE_PHI_GRID_j)   
    C_DEV.block_until_ready()               
    t1 = time.time()
    
    T_total = t1 - t0
    timing_results.append(T_total)
    
    if batch_size == Number_of_round:
        C_DATA = numpy.asarray(C_DEV)
    
    print(f"Batch size: {batch_size}, Timing: {T_total:.4f}s, Average: {T_total/batch_size:.6f}s")

In [ ]:
T_pyccl_average=[numpy.mean(T_pyccl[:batch_sizes[0]]),numpy.mean(T_pyccl[:batch_sizes[1]]),numpy.mean(T_pyccl[:batch_sizes[2]]),numpy.mean(T_pyccl[:batch_sizes[3]]),numpy.mean(T_pyccl[:batch_sizes[4]])]
pyplot.figure(figsize=(8,6))

pyplot.plot(batch_sizes, numpy.array(timing_results)/numpy.array(batch_sizes), color='orange', linestyle='--', linewidth=2.0)
pyplot.plot(batch_sizes, T_pyccl_average, color='black', linestyle='-', linewidth=2.0)

legend_handles = [
    mlines.Line2D([], [], color='black', linestyle='-', linewidth=2.5, label=r'$\mathrm{CCL}$'),
    mlines.Line2D([], [], color='orange', linestyle='--', linewidth=2.5, label=r'$\mathrm{TensorCloud-parallel}$'),
]
pyplot.legend(handles=legend_handles, fontsize=12)

pyplot.xlabel("Number of round", fontsize=14)
pyplot.ylabel("Average Execution time (s)", fontsize=14)

pyplot.grid(True, linestyle='--', alpha=0.5)
pyplot.tight_layout()
pyplot.savefig('speed-Y1.png')
pyplot.show()

In [ ]:
pyplot.figure(figsize=(8,6))

pyplot.plot(numpy.arange(Number_of_round), T_pyccl, color='black', linestyle='-', linewidth=2.0)
pyplot.hlines(y=T_total/Number_of_round, xmin=0, xmax=Number_of_round-1, color='orange', linestyle='--', linewidth=2.0)


legend_handles = [
    mlines.Line2D([], [], color='black', linestyle='-', linewidth=2.5, label=r'$\mathrm{CCL}$'),
    mlines.Line2D([], [], color='orange', linestyle='--', linewidth=2.5, label=r'$\mathrm{TensorCloud-parallel}$'),
]
pyplot.legend(handles=legend_handles, fontsize=12)

pyplot.xlabel("Number of round", fontsize=14)
pyplot.ylabel("Execution time (s)", fontsize=14)

pyplot.grid(True, linestyle='--', alpha=0.5)
pyplot.tight_layout()
pyplot.show()

In [ ]:
#Calculate the difference
C_DATA_mean=numpy.mean(C_DATA, axis=0)
C_CCL_mean=numpy.mean(C_CCL, axis=0)
C_RATIO = numpy.divide(C_DATA_mean, C_CCL_mean, out = numpy.ones(C_CCL_mean.shape), where = C_CCL_mean > 0)

In [ ]:
# Bin
with h5py.File(os.path.join(model_folder, '{}/TARGET/DATA0.hdf5'.format(tag)), 'r') as file:
    bin_lens = file['bin_lens'][...]
    bin_source = file['bin_source'][...]

bin_lens_size = len(bin_lens) - 1
bin_source_size = len(bin_source) - 1

In [ ]:
#Compare different bins
pyplot.rcParams['font.size'] = 24
pyplot.rcParams['text.usetex'] = True
FIGURE = pyplot.figure(figsize = (bin_source_size * 6, bin_source_size * 5))
GRIDSPEC = GridSpec(5 * bin_source_size, 6 * bin_source_size, figure = FIGURE, wspace = 0.0, hspace = 0.0)

for (BIN_INDEX1, BIN_INDEX2) in product(range(bin_source_size), range(bin_source_size)):
    
    PLOT = FIGURE.add_subplot(GRIDSPEC[5 * BIN_INDEX1: 5 * BIN_INDEX1 + 4, 6 * BIN_INDEX2: 6 * BIN_INDEX2 + 6])

    PLOT.plot(ell_grid, C_CCL_mean[BIN_INDEX1, BIN_INDEX2,:], color = 'black', linestyle = '-', linewidth = 2.5, label = r'$\mathrm{CCL-mean}$')
  
    PLOT.plot(ell_grid, C_DATA_mean[BIN_INDEX1, BIN_INDEX2,:], color = 'orange', linestyle = '-', linewidth = 2.5, label = r'$\mathrm{TensorCloud-mean}$')

    PLOT.text(x = 10**3, y = 10**-9, s = r'$m_1= {:.0f}$'.format(BIN_INDEX1 + 1), fontsize = 25)

    PLOT.text(x = 10**3, y = 10**-10, s = r'$m_2 = {:.0f}$'.format(BIN_INDEX2 + 1), fontsize = 25)

    PLOT.set_yscale('log')
    PLOT.set_xscale('log')
    PLOT.set_xlim(100, 2000)

    PLOT.set_ylim(10**-13, 10**-8)
    PLOT.yaxis.set_major_locator(FixedLocator([10**-8,10**-9,10**-10, 10**-11, 10**-12]))
    PLOT.yaxis.set_major_formatter(FixedFormatter([r'${10}^{-8}$',r'${10}^{-9}$', r'${10}^{-10}$', r'${10}^{-11}$', r'${10}^{-12}$']))
    if BIN_INDEX2 > 0: PLOT.set_yticklabels([])
    
    PLOT.set_xticklabels([])
    PLOT.get_yticklabels()[0].set_visible([])

    PLOT = FIGURE.add_subplot(GRIDSPEC[BIN_INDEX1 * 5 + 4: BIN_INDEX1 * 5 + 5, BIN_INDEX2 * 6: BIN_INDEX2 * 6 + 6])

    PLOT.plot(ell_grid, numpy.ones(ell_size + 1), color = 'black', linestyle = '-', linewidth = 2.5)

    PLOT.plot(ell_grid, numpy.ones(ell_size + 1) * (1.00 + 0.01), color = 'grey', linestyle = '--', linewidth = 2.5)

    PLOT.plot(ell_grid, numpy.ones(ell_size + 1) * (1.00 - 0.01), color = 'grey', linestyle = '--', linewidth = 2.5)
    
    PLOT.plot(ell_grid, C_RATIO[BIN_INDEX1, BIN_INDEX2,:], color = 'orange', linestyle = '-', linewidth = 2.5)

    PLOT.set_xscale('log')
    PLOT.set_xlim(100, 2000)
    PLOT.set_ylim(0.98, 1.02)
    PLOT.xaxis.set_major_locator(FixedLocator([100, 500, 1000]))
    PLOT.xaxis.set_major_formatter(FixedFormatter([r'$100$', r'$500$', r'${10}^3$']))

    if BIN_INDEX1 < bin_source_size - 1: PLOT.set_xticklabels([])
    if BIN_INDEX2 > 0: PLOT.set_yticklabels([])
    PLOT.get_xticklabels()[0].set_visible([])
legend_handles = [
    mlines.Line2D([], [], color='black', linestyle='-', linewidth=2.5, label=r'$\mathrm{CCL-mean}$'),
    mlines.Line2D([], [], color='orange', linestyle='-', linewidth=2.5, label=r'$\mathrm{TensorCloud-mean}$')
]

FIGURE.legend(handles=legend_handles, loc='upper left', fontsize=25)
FIGURE.suptitle(r'$C_{\epsilon \epsilon}^{ab} (\ell)$', fontsize=50, y=0.92)
FIGURE.supxlabel(r'$\ell$', fontsize = 50)
FIGURE.supylabel(r'$C_{\epsilon \epsilon}^{ab} (\ell)$', fontsize = 50)

FIGURE.subplots_adjust(hspace = 0.0, wspace = 0.0)
FIGURE.savefig('Cepseps-Y1.png')
FIGURE.show()